In [ ]:
import os
import numpy as np 
import cv2
import torch
import torch.nn as nn
from torchvision import models
from torch.autograd import Variable
from sklearn.decomposition import PCA

In [ ]:
def read_images(path, folders):
    images = []
    labels = []
    idx = 0
    for folder in folders:
        for filename in os.listdir(path+folder):
            image = cv2.imread(os.path.join(path+folder, filename))
            if image is not None:
                images.append(image)
                labels.append(idx)
                
        idx += 1
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

def getDescriptors(images, labels, model) : 
    features = []
    image_labels = []
    i = 0

    for image in images : 
        # Re-sizing the image
        image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)
        image = image.transpose(2, 1, 0)
        image = Variable(torch.Tensor(image))
        image = image.unsqueeze(0)
        feature = model(image)
        if feature is not None : 
            features.append(feature)
            image_labels.append(labels[i])
        i += 1
        print (feature)

        
    return features, image_labels

In [ ]:
model = models.alexnet(pretrained=True)
new_classifier = nn.Sequential(*list(model.classifier.children())[:-1])
model.classifier = new_classifier
model.eval()

In [ ]:
root = "/home/slr/Desktop/vr/Assignments/Feature-Learning/101_ObjectCategories/"

set_images = [ item for item in os.listdir(root) if os.path.isdir(os.path.join(root, item)) ]
images, labels = read_images(root, set_images)

In [ ]:
features, image_labels = getDescriptors(images, labels, model)

In [ ]:
pca = PCA(n_components = 0.9) 
features = pca.fit_transform(features) 

In [ ]:
# k = 50
# visDic = MiniBatchKMeans(init='k-means++',
#                             n_clusters=50,
#                             max_iter=1000,
#                             batch_size=1000, 
#                             n_init=10, 
#                             max_no_improvement=10, 
#                             verbose=0).fit(alexnet_des)


x_train, x_test, y_train, y_test = train_test_split(features, 
                                                    image_labels, 
                                                    test_size=0.1, 
                                                    random_state=4)


In [ ]:
clf = cv2.ml.KNearest_create()
clf.train(x_train, cv2.ml.ROW_SAMPLE, np.asarray(y_train, dtype=np.float32))

In [ ]:
ret, results, neighbours ,dist = clf.findNearest(x_test, k=10)

In [ ]:
pred_label = []
for var in results:
    label = var
    pred_label.append(int(label))

print (y_test)
print (pred_label)
    
# Measuring the accuracies
metrics.accuracy_score(y_test, pred_label)